In [19]:
import pandas as pd
import re
from helper.onemap import OneMapQuery
from tqdm import tqdm
import time


# Read the Excel file
df = pd.read_excel('store/data/travelling_salesman.xlsx')
df

,job_id,address
0,1,11 WOODLANDS INDUSTRIAL PARK E1 SINGAPORE 757734
1,2,58 JALAN TANAH PUTEH SINGAPORE 457360
2,3,97 YISHUN STREET 81 ORCHID PARK CONDOMINIUM SI...
3,4,11 CHIN BEE DRIVE SINGAPORE 619862
4,5,151 PUNGGOL CENTRAL NEIGHBOURHOOD POLICE CENTR...
5,6,1 ROBIN ROAD ONE ROBIN SINGAPORE 258176
6,7,23 KIM YAM ROAD WATERFORD RESIDENCE SINGAPORE ...
7,8,501 ORCHARD ROAD WHEELOCK PLACE SINGAPORE 238880
8,9,303 HENDERSON ROAD SAINT ANDREW'S NURSING HOME...
9,10,571 UPPER SERANGOON ROAD THE YARDLEY SINGAPORE...


In [20]:
def extract_postal_code(address: str) -> str:
    """Extract 6-digit postal code from address string"""
    # Find all sequences of 6 digits in the address
    matches = re.findall(r'\b\d{6}\b', address)
    return matches[0] if matches else None

# Extract postal codes and create new column
df['postal_code'] = df['address'].apply(extract_postal_code)
df

,job_id,address,postal_code
0,1,11 WOODLANDS INDUSTRIAL PARK E1 SINGAPORE 757734,757734
1,2,58 JALAN TANAH PUTEH SINGAPORE 457360,457360
2,3,97 YISHUN STREET 81 ORCHID PARK CONDOMINIUM SI...,768453
3,4,11 CHIN BEE DRIVE SINGAPORE 619862,619862
4,5,151 PUNGGOL CENTRAL NEIGHBOURHOOD POLICE CENTR...,828727
5,6,1 ROBIN ROAD ONE ROBIN SINGAPORE 258176,258176
6,7,23 KIM YAM ROAD WATERFORD RESIDENCE SINGAPORE ...,239333
7,8,501 ORCHARD ROAD WHEELOCK PLACE SINGAPORE 238880,238880
8,9,303 HENDERSON ROAD SAINT ANDREW'S NURSING HOME...,108925
9,10,571 UPPER SERANGOON ROAD THE YARDLEY SINGAPORE...,534798


In [21]:
# Get coordinates for each postal code with retry logic
def get_coordinates_with_retry(postal_code, max_retries=3):
    for attempt in range(max_retries):
        try:
            latlong = om.get_postal_latlong(postal_code)
            if latlong is not None:
                return latlong
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for postal code {postal_code}: {str(e)}")
            if attempt < max_retries - 1:  # Don't sleep on the last attempt
                print(f"Retrying in 2 seconds...")
                time.sleep(2)
    print(f"Failed to get coordinates for postal code {postal_code} after {max_retries} attempts")
    return None


# Initialize OneMap
om = OneMapQuery()

# Get coordinates for each postal code with progress bar
coordinates = []
for postal in tqdm(df['postal_code'], desc="Getting coordinates"):
    latlong = get_coordinates_with_retry(postal)
    coordinates.append(latlong)

# Add coordinates to dataframe
df['coordinates'] = coordinates

# Remove any rows where coordinates couldn't be obtained
df_clean = df.dropna(subset=['coordinates'])
print(f"\nSuccessfully retrieved coordinates for {len(df_clean)} out of {len(df)} locations")

# Display the results
df

Getting coordinates: 100%|██████████| 30/30 [00:12<00:00,  2.39it/s]


Successfully retrieved coordinates for 30 out of 30 locations


,job_id,address,postal_code,coordinates
0,1,11 WOODLANDS INDUSTRIAL PARK E1 SINGAPORE 757734,757734,"(1.45377086233108, 103.795470532339)"
1,2,58 JALAN TANAH PUTEH SINGAPORE 457360,457360,"(1.31638544752113, 103.93055168209)"
2,3,97 YISHUN STREET 81 ORCHID PARK CONDOMINIUM SI...,768453,"(1.41321614221681, 103.834841574765)"
3,4,11 CHIN BEE DRIVE SINGAPORE 619862,619862,"(1.3371550830213, 103.71522572957)"
4,5,151 PUNGGOL CENTRAL NEIGHBOURHOOD POLICE CENTR...,828727,"(1.39815002449975, 103.914054068345)"
5,6,1 ROBIN ROAD ONE ROBIN SINGAPORE 258176,258176,"(1.31891775628596, 103.828701817349)"
6,7,23 KIM YAM ROAD WATERFORD RESIDENCE SINGAPORE ...,239333,"(1.2940511158378, 103.839827515576)"
7,8,501 ORCHARD ROAD WHEELOCK PLACE SINGAPORE 238880,238880,"(1.30467628730252, 103.830642027577)"
8,9,303 HENDERSON ROAD SAINT ANDREW'S NURSING HOME...,108925,"(1.27092570676738, 103.812125318625)"
9,10,571 UPPER SERANGOON ROAD THE YARDLEY SINGAPORE...,534798,"(1.34770865960669, 103.871737392143)"


In [22]:
# Add depot location
depot_address = "CT Hub 2, 114 Lavender Street 338729"
depot_postal = "338729"
depot_latlong = om.get_postal_latlong(depot_postal)
print(f"Depot coordinates: {depot_latlong}")

Depot coordinates: (1.31158791334051, 103.863375124429)


In [23]:
# Create a list of unique locations including depot
all_locations = [depot_latlong]  # Start with depot as first location
location_to_index = {depot_latlong: 0}  # Mapping of coordinates to index
current_index = 1

# Create mapping for each location in df
location_mapping = []  # Will store index for each row in df
for coord in df['coordinates']:
    if coord not in location_to_index:
        location_to_index[coord] = current_index
        all_locations.append(coord)
        current_index += 1
    location_mapping.append(location_to_index[coord])

# Add mapping to dataframe
df['location_index'] = location_mapping
df

,job_id,address,postal_code,coordinates,location_index
0,1,11 WOODLANDS INDUSTRIAL PARK E1 SINGAPORE 757734,757734,"(1.45377086233108, 103.795470532339)",1
1,2,58 JALAN TANAH PUTEH SINGAPORE 457360,457360,"(1.31638544752113, 103.93055168209)",2
2,3,97 YISHUN STREET 81 ORCHID PARK CONDOMINIUM SI...,768453,"(1.41321614221681, 103.834841574765)",3
3,4,11 CHIN BEE DRIVE SINGAPORE 619862,619862,"(1.3371550830213, 103.71522572957)",4
4,5,151 PUNGGOL CENTRAL NEIGHBOURHOOD POLICE CENTR...,828727,"(1.39815002449975, 103.914054068345)",5
5,6,1 ROBIN ROAD ONE ROBIN SINGAPORE 258176,258176,"(1.31891775628596, 103.828701817349)",6
6,7,23 KIM YAM ROAD WATERFORD RESIDENCE SINGAPORE ...,239333,"(1.2940511158378, 103.839827515576)",7
7,8,501 ORCHARD ROAD WHEELOCK PLACE SINGAPORE 238880,238880,"(1.30467628730252, 103.830642027577)",8
8,9,303 HENDERSON ROAD SAINT ANDREW'S NURSING HOME...,108925,"(1.27092570676738, 103.812125318625)",9
9,10,571 UPPER SERANGOON ROAD THE YARDLEY SINGAPORE...,534798,"(1.34770865960669, 103.871737392143)",10


In [24]:
# Get route matrices for all locations (including depot)
duration_matrix, distance_matrix = om.get_route_matrices(all_locations)

print(f"\nTotal unique locations (including depot): {len(all_locations)}")
print(f"Depot is at index 0")


Total unique locations (including depot): 31
Depot is at index 0


In [25]:
duration_matrix

array([[   0., 2140., 1076., 1448., 1798., 1414.,  608.,  582.,  706.,
         983.,  571.,  237., 1046.,  850.,  663., 1034.,  729., 2582.,
        1897., 1232., 1493.,  497., 1358.,  744.,  703.,  379.,  602.,
         776., 1486., 1500.,  646.],
       [2140.,    0., 2722., 1040., 1973., 1825., 1871., 2295., 2023.,
        2783., 1851., 2084., 1419., 2020., 2223., 2335., 2402., 2784.,
         468., 1652., 2008., 2308., 2087., 2232., 1756., 2227., 2273.,
        2331., 2104., 2435., 2030.],
       [1076., 2722.,    0., 2118., 2340., 1491., 1415., 1279., 1527.,
        1510., 1265., 1244., 1858., 1282.,  927., 1685.,  509., 3135.,
        2570., 1796., 2085., 1218., 1023., 1455., 1342., 1152., 1308.,
         534., 2054., 1147., 1435.],
       [1448., 1040., 2118.,    0., 2176., 1202., 1566., 1603., 1688.,
        2091., 1247., 1480.,  861., 1416., 1532., 2077., 1710., 2747.,
         665., 1048., 1961., 1704., 1385., 1777., 1065., 1623., 1610.,
        1727., 2067., 1744., 1628.],


In [26]:
from vroom import Vehicle, Job, Input

# Create VROOM input
vroom_input = Input()

# Create vehicle starting and ending at depot (index 0)
vehicle = Vehicle(
    id=1,  # Vehicle ID
    start=0,  # Depot location index
    end=0  # Return to depot
)
vroom_input.add_vehicle(vehicle)

In [27]:
# Create jobs from the dataframe
for _, row in df.iterrows():
    job = Job(
        id=row['job_id'],  # Use job_id from dataframe
        location=row['location_index']  # Use location_index we created
    )
    vroom_input.add_job(job)


In [28]:
# Set the matrices in the input
vroom_input.set_durations_matrix(
    profile="car",
    matrix_input=duration_matrix
)
vroom_input.set_distances_matrix(
    profile="car",
    matrix_input=distance_matrix
)

In [29]:
# Solve the problem
solution = vroom_input.solve(exploration_level=5, nb_threads=4)

print("\nOptimization Results:")
print(f"Total duration: {solution.summary.duration}")
print(f"Total distance: {solution.summary.distance}")


Optimization Results:
Total duration: 18810
Total distance: 160531


In [30]:
solution.routes

,vehicle_id,type,arrival,duration,setup,service,waiting_time,location_index,id,description,distance
0,1,start,0,0,0,0,0,0,<NA>,,0
1,1,job,663,663,0,0,0,14,14,,5293
2,1,job,1199,1199,0,0,0,27,27,,9166
3,1,job,1461,1461,0,0,0,16,16,,10668
4,1,job,1970,1970,0,0,0,2,2,,14321
5,1,job,3117,3117,0,0,0,29,29,,24112
6,1,job,3891,3891,0,0,0,22,22,,31074
7,1,job,4503,4503,0,0,0,5,5,,36414
8,1,job,5110,5110,0,0,0,19,19,,41475
9,1,job,5940,5940,0,0,0,13,13,,47815


In [31]:
# Convert solution routes to a dataframe for easier analysis
route_df = solution.routes.copy()

# Get all job IDs from the solution (excluding start/end rows)
completed_jobs = route_df[route_df['type'] == 'job']['id'].tolist()

# Get all job IDs from our original dataframe
all_jobs = df['job_id'].tolist()

# Check for missing jobs
missing_jobs = set(all_jobs) - set(completed_jobs)

if len(missing_jobs) == 0:
    print("All jobs were fulfilled!")
   
else:
    print("Warning: Some jobs were not fulfilled!")
    print("Missing jobs:", sorted(list(missing_jobs)))

All jobs were fulfilled!


In [32]:
import folium
# Create a map centered on Singapore
m = folium.Map(location=[1.352083, 103.819839], zoom_start=12, tiles="cartodbpositron")

# Get the sequence of locations from the solution
route_sequence = []
for step in solution.routes.itertuples():
    if step.type == 'start':
        route_sequence.append(all_locations[step.location_index])
    elif step.type == 'job':
        route_sequence.append(all_locations[step.location_index])
    elif step.type == 'end':
        route_sequence.append(all_locations[step.location_index])

In [33]:
route_sequence

[(1.31158791334051, 103.863375124429),
 (1.33631557522577, 103.888825620348),
 (1.32584906783406, 103.909184600518),
 (1.31742494084104, 103.910444979932),
 (1.31638544752113, 103.93055168209),
 (1.35516120754023, 103.967844585142),
 (1.36857369907868, 103.945571141969),
 (1.39815002449975, 103.914054068345),
 (1.39471047297983, 103.880244100805),
 (1.36197570492975, 103.881207403589),
 (1.34770865960669, 103.871737392143),
 (1.34770718525067, 103.859409321337),
 (1.36955714917919, 103.827657960529),
 (1.41321614221681, 103.834841574765),
 (1.44261710306115, 103.816962441559),
 (1.45377086233108, 103.795470532339),
 (1.31586685970156, 103.630129366512),
 (1.3371550830213, 103.71522572957),
 (1.33329506563598, 103.742153884191),
 (1.32783798911958, 103.747950360251),
 (1.29385171138219, 103.78442753238),
 (1.27092570676738, 103.812125318625),
 (1.29102791800182, 103.825340807968),
 (1.30467628730252, 103.830642027577),
 (1.31891775628596, 103.828701817349),
 (1.30167305661383, 103.83002

In [34]:
# Add depot marker with special styling
folium.CircleMarker(
    location=depot_latlong,
    popup='Depot (CT Hub 2)',
    tooltip='Depot',
    color='red',
    fill=True,
    fillColor='red',
    radius=10
).add_to(m)

# Plot routes between consecutive points using same color for vehicle
vehicle_color_index = 0  # Use first color for our single vehicle
for i in range(len(route_sequence)-1):
    start = route_sequence[i]
    end = route_sequence[i+1]
    # Plot route with sequence numbers, using same color index
    om.plot_routes(start, end, m, vehicle_color_index + 1, (i, i+1))


In [35]:
m

In [36]:
m.save('optimized_route.html')

print("Map has been saved as 'travelling_salesman_route.html'")
print(f"Total stops: {len(route_sequence)-2}")  # Subtract depot start and end

Map has been saved as 'travelling_salesman_route.html'
Total stops: 30
